In [1]:
import random
from classes.minion import Minion
from classes.hero import Hero
from scenarios.deck_1 import deck  # Example deck import
from solver.slv import run_single_turn
from state.transition_state import start_turn, apply_results, end_turn

In [2]:
friendly_minions = [
    Minion(name="River Crocolisk", minion_class="Neutral", keywords=[], attack=2, health=3, strat_value=1, mana_cost=2),
    Minion(name="Silverback Patriarch", minion_class="Neutral", keywords=["Taunt"], attack=1, health=4, strat_value=1, mana_cost=3),
    Minion(name="Chillwind Yeti", minion_class="Neutral", keywords=[], attack=4, health=5, strat_value=1, mana_cost=4),
    Minion(name="Kor'kron Elite", minion_class="Warrior", keywords=["Charge"], attack=4, health=3, strat_value=1, mana_cost=4),
]

enemy_minions = [
    Minion(name="Silver Hand Recruit", minion_class="Paladin", keywords=[], attack=1, health=1, strat_value=1, mana_cost=1),
    Minion(name="Righteous Protector", minion_class="Paladin", keywords=["Taunt", "Divine Shield"], attack=1, health=1, strat_value=1, mana_cost=1),
    Minion(name="Oasis Snapjaw", minion_class="Neutral", keywords=[], attack=2, health=7, strat_value=1, mana_cost=4),
]

# Renamed from "hand" to "hand_example"
hand_list = [
    Minion(name="Murloc Raider", minion_class="Neutral", keywords=[], attack=2, health=1, strat_value=1, mana_cost=1),
    Minion(name="Bloodfen Raptor", minion_class="Neutral", keywords=[], attack=3, health=2, strat_value=1, mana_cost=2),
    Minion(name="Kor'kron Elite", minion_class="Warrior", keywords=["Charge"], attack=4, health=3, strat_value=2, mana_cost=4),
]

In [3]:
def main():
    # 1) Create Hero objects (assuming a Hero class exists)
    hero1 = Hero("Paladin", 30)
    hero2 = Hero("Warrior", 30)

    # 2) Print the deck size before drawing
    print(f"Deck has {len(deck)} cards at the start.")

    # 3) Start turn: hero1 draws exactly one card from the deck
    start_turn(hero1, deck, hand_list)

    # Check what got drawn
    print(f"After draw, hand_list has {len(hand_list)} cards.")
    for k, card in enumerate(hand_list):
        print(f"  Hand card {k}: {card.name} "
              f"(Atk={card.attack}, HP={card.health}, Cost={card.mana_cost})")

    # 4) Build arrays for the solver
    m = len(friendly_minions)   # number of friendly minions on board
    n = len(enemy_minions)      # number of enemy minions on board
    h = len(hand_list)          # number of cards in hand
    M = 5                       # Mana for this turn
    H_hero = 12                  # Opponent hero health

    # ---- Print the counts and minion details ----
    print(f"Number of friendly minions on board: {m}")
    for i, fmn in enumerate(friendly_minions):
        print(f"  - Friendly Minion {i}: {fmn.name} "
              f"(Atk={fmn.attack}, HP={fmn.health})")

    print(f"Number of enemy minions on board: {n}")
    for j, emn in enumerate(enemy_minions):
        print(f"  - Enemy Minion {j}: {emn.name} "
              f"(Atk={emn.attack}, HP={emn.health})")

    print(f"Number of cards in hand: {h}")
    for k, card in enumerate(hand_list):
        print(f"  - Hand Card {k}: {card.name} "
              f"(Atk={card.attack}, HP={card.health}, Cost={card.mana_cost})")

    # Combine board minions + hand minions for indexing in the solver
    combined = friendly_minions + hand_list
    A = [mn.attack for mn in combined]   # attack values
    B = [mn.health for mn in combined]   # health values
    P = [enm.attack for enm in enemy_minions]
    Q = [enm.health for enm in enemy_minions]

    C = [mn.mana_cost for mn in hand_list]
    S = [mn.strat_value for mn in hand_list]

    # 5) Define or load your weights
    weights = {
        "W1": 10,  # e.g. hero alive
        "W2": 9,   # board clear
        "W3": 8,   # minion->hero damage
        "W4": 7,   # penalty for leaving strong enemy minion
        "W5": 6,   # keep friendly minion health
        "W6": 5,   # minion->minion attack
        "W7": 4,   # penalty for taking damage from enemy minions
        "W8": 3    # strategic card plays
    }

    # 6) Solve (call the new run_single_turn with a weights dict)
    solution = run_single_turn(
        m=m, n=n, h=h,
        M=M, H_hero=H_hero,
        A=A, B=B, P=P, Q=Q, C=C, S=S,
        weights=weights
    )

    # 7) Apply results to scenario_data
    scenario_data = {
        "A": A,
        "B": B,
        "P": P,
        "Q": Q,
        "H_hero": H_hero,
    }
    apply_results(solution, scenario_data)

    # 8) End turn
    end_turn(hero1, hero2)
    print("Turn complete. Inspect logs above for solver outcome.")


if __name__ == "__main__":
    main()


Deck has 5 cards at the start.
Paladin draws: Minion: Wolfrider | Class: Neutral | Attack: 3 | Health: 1 | Keywords: Charge
After draw, hand_list has 4 cards.
  Hand card 0: Murloc Raider (Atk=2, HP=1, Cost=1)
  Hand card 1: Bloodfen Raptor (Atk=3, HP=2, Cost=2)
  Hand card 2: Kor'kron Elite (Atk=4, HP=3, Cost=4)
  Hand card 3: Wolfrider (Atk=3, HP=1, Cost=3)
Number of friendly minions on board: 4
  - Friendly Minion 0: River Crocolisk (Atk=2, HP=3)
  - Friendly Minion 1: Silverback Patriarch (Atk=1, HP=4)
  - Friendly Minion 2: Chillwind Yeti (Atk=4, HP=5)
  - Friendly Minion 3: Kor'kron Elite (Atk=4, HP=3)
Number of enemy minions on board: 3
  - Enemy Minion 0: Silver Hand Recruit (Atk=1, HP=1)
  - Enemy Minion 1: Righteous Protector (Atk=1, HP=1)
  - Enemy Minion 2: Oasis Snapjaw (Atk=2, HP=7)
Number of cards in hand: 4
  - Hand Card 0: Murloc Raider (Atk=2, HP=1, Cost=1)
  - Hand Card 1: Bloodfen Raptor (Atk=3, HP=2, Cost=2)
  - Hand Card 2: Kor'kron Elite (Atk=4, HP=3, Cost=4)
  -